# DVS Gesture

Add intro...

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from lava.magma.core.run_configs import Loihi2HwCfg
from lava.magma.core.run_conditions import RunSteps
from lava.proc import io
from lava.proc import snip_io as sio

from dataset import IBMGestureDataset

## Network Exchange (NetX) Libarary

NetX api allows automatic creation of Lava process from the network specification. It is available as a part of Lava-DL library as `lava.lib.dl.netx`

In [2]:
from lava.lib.dl import netx
from lava.lib.dl import slayer

## Create network block

A Lava process describing the network can be created by simply instantiating `netx.hdf5.Network` with the path of the desired hdf5 network description file.
* The input layer is accessible as `net.in_layer`.
* The output layer is accessible as `net.out_layer`.
* All the constituent layers are accessible as as a list: `net.layers`.

In [3]:
net = netx.hdf5.Network(net_config='Trained/network.net')
print(net)

|   Type   |  W  |  H  |  C  | ker | str | pad | dil | grp |delay|
|Conv      |   16|   16|   16| 5, 5| 2, 2| 2, 2| 1, 1|    1|False|
|Conv      |    8|    8|   32| 3, 3| 2, 2| 1, 1| 1, 1|    1|False|
|Dense     |    1|    1|  512|     |     |     |     |     |False|
|Dense     |    1|    1|   11|     |     |     |     |     |False|


In [4]:
num_samples = 20
steps_per_sample = 1500
blank_time = 100
num_steps = num_samples * (steps_per_sample + blank_time) + 1

## Create Dataset instance
Typically the user would write it or provide it.

In [5]:
test_set = IBMGestureDataset(path='/nfs/ncl/datasets/DVSgesture',
                             steps_per_sample=steps_per_sample,
                             blank_time=blank_time)

# Instantiate Dataloader

In [6]:
dataloader = io.dataloader.SpikeDataloader(dataset=test_set,
                                           interval=steps_per_sample + blank_time)

# Connect Input and Output

In [7]:
gt_logger = io.sink.RingBuffer(shape=(1,), buffer=num_steps)
output_logger = io.sink.RingBuffer(shape=net.out_layer.out.shape,
                                   buffer=num_steps)
inp_adapter = sio.spike.PyToNxAdapter(shape=net.inp.shape)
out_adapter = sio.spike.NxToPyAdapter(shape=net.out.shape)

dataloader.ground_truth.connect(gt_logger.a_in)
dataloader.s_out.connect(inp_adapter.inp)
inp_adapter.out.connect(net.inp)
net.out.connect(out_adapter.inp)
out_adapter.out.connect(output_logger.a_in)

## Run the network


In [8]:
import os
import logging

os.environ['SLURM'] = '1'
os.environ['PARTITION'] = 'oheogulch'
os.environ['LOIHI_GEN'] = 'N3B3'

In [9]:
from lava.proc.conv.process import Conv
from lava.proc.conv.ncmodels import NcL2ModelConv, NcL1ModelConvAsSparse

class CustomRunCfg(Loihi2HwCfg):
    def select(self, process, proc_models):
        if isinstance(process, Conv):
            return NcL1ModelConvAsSparse
        else:
            return super().select(process, proc_models)

In [10]:

net._log_config.level = logging.INFO
net.run(condition=RunSteps(num_steps=num_steps), run_cfg=CustomRunCfg())
output = output_logger.data.get()
net.stop()

Violation core_id=1 reg_name='SynMem' allocation=247808 self.cost_db.registers[reg_name]=8184
Violation core_id=2 reg_name='SynMem' allocation=32048 self.cost_db.registers[reg_name]=8184
Violation core_id=4 reg_name='SynMem' allocation=11664 self.cost_db.registers[reg_name]=8184
Final max_ratio=1
Final max_ratio=31
Violation core_id=1 reg_name='SynMem' allocation=16384 self.cost_db.registers[reg_name]=8184
Final max_ratio=5
Final max_ratio=2
Final max_ratio=1
Final max_ratio=3
Final max_ratio=1
Final max_ratio=1


SpikeCounterLimitError: The number of spike counters per embedded core is currently limited to 900. Requested number of spike counters: 2048.

In [ ]:
test_set[0][0].shape

(32, 32, 2, 1600)

In [ ]:
net.inp.shape

(32, 32, 2)

In [ ]:
out_adapter.inp.shape

(11,)